# Import

In [1]:
!pip install --upgrade -q tensorflow
!pip install --upgrade -q sagemaker
!pip install --upgrade -q boto3
!pip install --upgrade -q mlflow

In [24]:
import os

import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

import mlflow

from sagemaker.tensorflow.estimator import TensorFlow
from sagemaker.tensorflow.model import TensorFlowModel

# AWS Setting

In [3]:
sagemaker_session = Session(default_bucket="mlflow-tracing-dir")
boto_session = boto3.Session()

role = get_execution_role()
default_bucket = sagemaker_session.default_bucket

sagemaker_client = boto_session.client("sagemaker")
region = boto_session.region_name

In [4]:
print(
    f"SageMaker Session : {sagemaker_session}",
    f"\nBoto3 Session : {boto_session}",
    f"\nExecution Role : {role}",
    f"\nDefault Bucket : {default_bucket}",
    f"\nSageMaker Client Instance : {sagemaker_client}",
    f"\nCurrent Region : {region}"
)

SageMaker Session : <sagemaker.session.Session object at 0x7fc7d4d01a60> 
Boto3 Session : Session(region_name='ap-southeast-2') 
Execution Role : arn:aws:iam::954690186719:role/service-role/AmazonSageMaker-ExecutionRole-20250821T100350 
Default Bucket : <bound method Session.default_bucket of <sagemaker.session.Session object at 0x7fc7d4d01a60>> 
SageMaker Client Instance : <botocore.client.SageMaker object at 0x7fc7cfbb5c70> 
Current Region : ap-southeast-2


# Define & Train Estimator

In [5]:
EXPERIMENT_NAME = "tensorflow-estimator-lab"
RUN_NAME = "tensorflow-exp-1"

In [6]:
PARAMETERS = {
    "batch_size" : 256,
    "epochs" : 5,
    "dropout" : 0.5,
    "experiment_name" : EXPERIMENT_NAME,
    "run_name" : RUN_NAME
}

ESTIMATOR_CFG = {
    "entry_point" : "./script/train.py",
    "role" : role,
    "region" : region
}

In [19]:
class EstimatorTrainer:
    def __init__(self, estimator_cfg, param_set):
        # create estimator
        self.estimator = TensorFlow(
            entry_point=estimator_cfg['entry_point'],
            role=estimator_cfg['role'],
            framework_version="2.8",
            py_version="py39",
            instance_type="ml.m5.xlarge",
            instance_count=1,
            # keep_alive_period_in_seconds=3600,
            environment={"REGION" : estimator_cfg['region']},
            model_dir=False,
            hyperparameters=param_set
        )

    def fit(self):
        # train estimator (create training job)
        self.estimator.fit()

In [22]:
trainer = EstimatorTrainer(
    estimator_cfg=ESTIMATOR_CFG,
    param_set=PARAMETERS
)

In [23]:
trainer.fit()

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2025-08-22-05-49-33-239


2025-08-22 05:49:33 Starting - Starting the training job...
2025-08-22 05:50:05 Starting - Preparing the instances for training...
2025-08-22 05:50:22 Downloading - Downloading input data...
2025-08-22 05:50:42 Downloading - Downloading the training image...
2025-08-22 05:51:23 Training - Training image download completed. Training in progress..2025-08-22 05:51:33.311995: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-08-22 05:51:33.312171: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-08-22 05:51:33.341193: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-08-22 05:51:35,114 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-08-22 05:51:35,123 sagemaker-training-toolkit INFO     N

# Model Registry

In [25]:
model = TensorFlowModel(model_data=trainer.estimator.model_data, role=role, framework_version="2.8")

In [26]:
model.register(
    model_package_group_name="tensorflow-estimator-lab-model-1",
    content_types=["text/csv"],
    inference_instances=["ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    response_types=["text/csv"],
    approval_status="PendingManualApproval"
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
